# global ranking monitoring for LANEIGE (data from @COSME)

본 코드는 @COSME 사이트의 화장품 랭킹 정보를 추출하는 코드로 변수명을 일본어(홈페이지 원어) 기준으로 코드를 작성함.

## Import

In [1]:
import requests
import re
import time
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
import os

from tqdm import tqdm

## 데이터 수집 날짜 기록

랭킹은 매주 금요일에 업데이트 되기 때문에, 크롤링 로직이 `매주 금요일`에 실행되도록 자동화

In [2]:
global collect_date
collect_date = datetime.today().strftime("%Y/%m/%d")
print("데이터 수집 날짜: ", collect_date)

데이터 수집 날짜:  2025/12/08


## COSME 카테고리별 랭킹 데이터 크롤링

### 카테고리 생성

| url 주소 기반 카테고리 코드 | 카테고리명 |
|-----------------------|----------|
| total | 종합뷰티랭킹|
| 800 | 종합스킨케어 |
| 1005 | 페이셜크림 |
| 904 | 페이셜마스크 |
| 803 | 베이스메이크업 |


<br>

\* _모니터링 대상 제품_

- 아토코스메(종합 뷰티 랭킹)
- 아토코스메(종합 스킨케어)
- 아토코스메(페이셜크림)
- 아토코스메(페이셜마스크)
- 아토코스메(베이스메이크업)

In [3]:
global category_name

category_name = {"total":"종합뷰티랭킹", 800: "종합스킨케어", 1005: "페이셜크림", 904: "페이셜마스크", 803: "베이스메이크업"}

### 종합 뷰티 랭킹

수집 대상 데이터

- `start_date ~ end_date` : 랭킹 기간(시작 날짜 ~ 종료 날짜)

- `rank` : 제품 순위

- `brand` : 브랜드명

- `product_name` : 제품명

- `release_date` : 제품 발매일

In [4]:
def extract_cosme_laneige_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []

    # 1. 랭킹 기간 추출
    date_block = soup.select_one("div#nav-rank-footer p")

    if date_block:
        text = date_block.text.strip()
        match = re.search(r"(\d{4}/\d{1,2}/\d{1,2})\s*[〜～-]\s*(\d{4}/\d{1,2}/\d{1,2})", text)
        start_date = match.group(1) if match else None
        end_date = match.group(2) if match else None
    else:
        start_date, end_date = None, None

    # 2. brand 필터 설정
    brand_name = "LANEIGE(ラネージュ)"
    #brand_name = "SHISEIDO" # test용

    # 3. 제품 정보 데이터 추출
    dl_list = soup.select("div#list-item dl")

    for dl in dl_list:

        # 3.1. 순위 추출
        rank_num_block = dl.select_one("span.rank-num")
        if not rank_num_block:
            continue

        img_tag = rank_num_block.select_one("img[alt]")

        # 상위 3위는 1위, 2위, 3위로 되어 있기 때문에 alt 속성에서 숫자 추출
        if img_tag:
            alt_text = img_tag.get("alt", "")
            numbers = re.findall(r"\d+", alt_text)
            rank = int(numbers[0]) if numbers else None

        # 4위 이하부터는 일반 텍스트로 되어 있기 때문에 텍스트에서 숫자 추출
        else:
            num_tag = rank_num_block.select_one("span.num")
            if not num_tag:
                continue
            numbers = re.findall(r"\d+", num_tag.text.strip())
            rank = int(numbers[0]) if numbers else None

        if rank is None:
            continue

        # 3.2. 브랜드명 추출
        brand_tag = dl.select_one("dd.summary span.brand a")
        if not brand_tag:
            continue

        brand = brand_tag.text.strip()
        if not brand_name in brand:
            continue

        # 3.3. 제품명 추출
        item_tag = dl.select_one("dd.summary span.item a")
        product_name = item_tag.text.strip() if item_tag else None

        # 3.4. 제품 발매일 추출
        release_tag = dl.select_one("p.onsale")
        if release_tag:
            release_text = release_tag.text.strip()
            match_release = re.search(r"発売日：(\d{4}/\d{1,2}/\d{1,2})", release_text)
            release_date = match_release.group(1) if match_release else None
        else:
            release_date = None

        # 추출한 데이터
        result.append({
            "collect_date": collect_date,
            "start_date": start_date,
            "end_date": end_date,
            "rank": rank,
            "brand": brand,
            "product_name": product_name,
            "release_date": release_date,
        })

    return result

# ---------------------------------------------------
def total_beauty_ranking():
    base_url = "https://www.cosme.net/ranking/products/page/{}"
    all_results = []

    for page in tqdm(range(1, 10)): # 종합 뷰티 랭킹은 1위 ~ 100위까지 (총 10페이지))
        url = (
            "https://www.cosme.net/ranking/products/"
            if page == 1
            else base_url.format(page)
        )

        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if res.status_code != 200:
            print(f"페이지 로드 실패: {url}")
            continue

        page_results = extract_cosme_laneige_from_html(res.text)
        all_results.extend(page_results)

        time.sleep(1) # 서버 부하 방지

    return all_results

laneige_data = total_beauty_ranking()

100%|██████████| 9/9 [00:20<00:00,  2.26s/it]


In [5]:
# 종합 뷰티 랭킹 데이터 저장
data_total = pd.DataFrame(laneige_data)
data_total

""


### 종합 뷰티 랭킹 이외 카테고리

[종합 스킨케어, 페이셜크림, 페이셜마스크, 베이스메이크업] 에 해당하는 카테고리는 모두 동일한 페이지 형식을 가지고 있어서, 카테고리별 url만 구분하여 하나의 크롤링 코드를 활용함

수집 대상 데이터 (앞서 크롤링한 `종합 뷰티 랭킹` 수집 대상 데이터와 동일)

- `start_date ~ end_date` : 랭킹 기간(시작 날짜 ~ 종료 날짜)

- `rank` : 제품 순위

- `brand` : 브랜드명

- `product_name` : 제품명

- `release_date` : 제품 발매일

In [6]:
# 동일 코드 반복 방지를 위한 함수화

def extract_cosme_laneige_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []

    # 1. 랭킹 기간 추출
    date_block = soup.select_one("div#keyword-ranking-header p")

    if date_block:
        text = date_block.text.strip()
        match = re.search(r"(\d{4}/\d{1,2}/\d{1,2})\s*[〜～-]\s*(\d{4}/\d{1,2}/\d{1,2})", text)
        start_date = match.group(1) if match else None
        end_date = match.group(2) if match else None
    else:
        start_date, end_date = None, None

    # 2. brand 필터 설정
    brand_name = "LANEIGE(ラネージュ)"
    #brand_name = "SHISEIDO" # test용

    # 3. 제품 정보 데이터 추출
    dl_list = soup.select("div#keyword-ranking-list dl")

    for dl in dl_list:

        # 3.1. 순위 추출
        rank_num_block = dl.select_one("span.rank-num")
        if not rank_num_block:
            continue

        img_tag = rank_num_block.select_one("img[alt]")
        if img_tag:
            alt_text = img_tag.get("alt", "")
            numbers = re.findall(r"\d+", alt_text)
            rank = int(numbers[0]) if numbers else None
        else:
            num_tag = rank_num_block.select_one("span.num")
            if not num_tag:
                continue
            numbers = re.findall(r"\d+", num_tag.text.strip())
            rank = int(numbers[0]) if numbers else None

        if rank is None:
            continue

        # 3.2. 브랜드명 추출
        brand_tag = dl.select_one("dd.summary span.brand a")
        if not brand_tag:
            continue

        brand = brand_tag.text.strip()
        if not brand_name in brand:
            continue

        # 3.3. 제품명 추출
        item_tag = dl.select_one("dd.summary h4.item a")
        product_name = item_tag.text.strip() if item_tag else None

        # 3.4. 제품 발매일 추출
        release_tag = dl.select_one("p.onsale")
        if release_tag:
            release_text = release_tag.text.strip()
            match_release = re.search(r"発売日：(\d{4}/\d{1,2}/\d{1,2})", release_text)
            release_date = match_release.group(1) if match_release else None
        else:
            release_date = None

        # 추출한 데이터
        result.append({
            "collect_date": collect_date,
            "start_date": start_date,
            "end_date": end_date,
            "rank": rank,
            "brand": brand,
            "product_name": product_name,
            "release_date": release_date,
        })

    return result


# ---------------------------------------------------
# 카테고리 코드별로 크롤링

def category_ranking(category_id):
    base_url = f"https://www.cosme.net/categories/item/{category_id}/ranking/?page="
    all_results = []
    print(f"Data Crawling: category_name = {category_name[category_id]}")
    for page in tqdm(range(1, 6)): # 랭킹은 1위 ~ 50위까지 (총 5페이지))
        url = (
            f"https://www.cosme.net/categories/item/{category_id}/ranking/"
            if page == 1
            else base_url+str(page)
        )

        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if res.status_code != 200:
            print(f"페이지 로드 실패: {url}")
            continue

        page_results = extract_cosme_laneige_from_html(res.text)
        all_results.extend(page_results)

        time.sleep(1) # 서버 부하 방지

    return all_results


if __name__ == "__main__":
    category_ids = list(category_name.keys())[1:]
    collect_date = datetime.today().strftime("%Y/%m/%d")
    for category_id in category_ids:
        laneige_data = category_ranking(category_id)
        globals()[f'data_{category_id}'] = pd.DataFrame(laneige_data)

Data Crawling: category_name = 종합스킨케어


100%|██████████| 5/5 [00:13<00:00,  2.74s/it]


Data Crawling: category_name = 페이셜크림


100%|██████████| 5/5 [00:15<00:00,  3.17s/it]


Data Crawling: category_name = 페이셜마스크


100%|██████████| 5/5 [00:12<00:00,  2.60s/it]


Data Crawling: category_name = 베이스메이크업


100%|██████████| 5/5 [00:13<00:00,  2.78s/it]


## 엑셀 파일에 데이터 저장

`COSME_LANEIGE_Ranking_History.xlsx` 파일에 카테고리별 데이터를 저장하거나, 이미 존재하는 경우 기존 시트 아래에 새로운 데이터를 추가함.

---

1. **파일 존재 여부 확인**
- 엑셀 파일이 없으면 새로 생성
- 이미 존재하면 기존 파일에 데이터를 추가

---

2. **(초기)파일 생성이 필요한 경우**
- ExcelWriter를 `mode='w'`로 열어 엑셀 파일을 생성합
- `category_name`에 포함된 각 카테고리에 대해:
  - 전역 변수(data_카테고리명)에서 크롤링한 데이터 가져옴
  - 데이터가 없거나 비어 있으면 지정된 columns로 빈 DataFrame 생성함
  - 해당 시트 이름으로 파일에 기록

---

3. **파일이 존재할 경우**
- ExcelWriter를 `mode='a'`로 열어 파일에 추가 기록함
- `if_sheet_exists='overlay'` 옵션으로 기존 시트를 유지
- `category_name`에 포함된 각 카테고리에 대해:
  - 전역 변수(data_카테고리명)에서 크롤링한 데이터 가져오고, 비어 있으면 skip
  - `header=False`로 설정하여 헤더 없이 기존 데이터에 이어서 추가



In [8]:
file_path = 'data/COSME_LANEIGE_Ranking_History.xlsx'
columns = ["collect_date", "start_date", "end_date", "rank", "brand", "product_name", "release_date"]

if not os.path.exists(file_path):

    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        for name in category_name:
            df = globals().get(f'data_{name}', None)

            if df is None or len(df) == 0:
                df = pd.DataFrame(columns=columns)

            df.to_excel(writer, sheet_name=category_name[name], index=False)

else:

    with pd.ExcelWriter(
        file_path,
        engine='openpyxl',
        mode='a',
        if_sheet_exists='overlay'
    ) as writer:

        for name in category_name:
            sheet_name = category_name[name]
            df = globals().get(f'data_{name}', None)

            if df is None or len(df) == 0:
                continue

            start_row = writer.sheets[sheet_name].max_row

            df.to_excel(
                writer,
                sheet_name=sheet_name,
                startrow=start_row,
                index=False,
                header=False
            )


### 엑셀 파일로 저장한 크롤링 데이터 가져오기

Example>

앞선 크롤링 코드에서
1. Brand: Anua의 카테고리별 랭킹 내 데이터를 전부 불러오고 -> brand_name = "Anua"
2. Brand: SHISEIDO의 데이터를 전부 불러옴. -> brand_name = "SHISEIDO"

=> brand_name에 할당된 변수만 변경해서 데이터 불러옴.

**목적**: 원하는대로 크롤링 잘 해오는지 확인 + 엑셀 파일 새로 생성, 기존 엑셀 파일에 데이터 추가 확인을 위해

In [9]:
from IPython.display import display, HTML


for cate in category_name:
    data = pd.read_excel(file_path, sheet_name=category_name[cate])
    print("category name: ",category_name[cate])
    display(data)
    print("---------------------")

category name:  종합뷰티랭킹


,collect_date,start_date,end_date,rank,brand,product_name,release_date


---------------------
category name:  종합스킨케어


,collect_date,start_date,end_date,rank,brand,product_name,release_date


---------------------
category name:  페이셜크림


...

---------------------
category name:  페이셜마스크


,collect_date,start_date,end_date,rank,brand,product_name,release_date


---------------------
category name:  베이스메이크업


,collect_date,start_date,end_date,rank,brand,product_name,release_date


---------------------


<br>

2025년 12월 06일 수집 기준. 인사이트를 도출할 데이터가 매우 부족하다.

데이터 증강 필요

1. @COSME에서 라네즈 데이터 가져오기(카테고리별_종합뷰티랭킹 제외)
2. 1로 가져온 데이터를 기반으로 임의로 데이터 추가해서 데이터 증강
    - `cosme_LANEIGE_data_augmentation.ipynb` → Data Augmentation

In [ ]:
def extract_cosme_laneige_product_info_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []

    # 3. 제품 정보 데이터 추출
    dl_list = soup.select("div.info")
    for dl in dl_list:


        # 3.2. 브랜드명 추출
        brand = "LANEIGE(ラネージュ)"

        # 3.3. 제품명 추출
        product_name_tag = dl.select_one("h2 > a:first-of-type")
        product_name = product_name_tag.text.strip() if product_name_tag else None

        # 3.4. 제품 발매일 추출
        release_tag = dl.select_one("ul.desc")
        if release_tag:
            release_text = release_tag.text.strip()
            match_release = re.search(r"発売日：(\d{4}/\d{1,2}/\d{1,2})", release_text)
            release_date = match_release.group(1) if match_release else None
        else:
            release_date = None

        # 추출한 데이터
        if release_date:
            result.append({

                "brand": brand,
                "product_name": product_name,
                "release_date": release_date,
            })

    return result

In [ ]:
def product_info():
    base_url = "https://cosmeet.cosme.net/product/search/page/{}/srt/4/fw/laneige/itm/800"
    all_results = []
    for page in tqdm(range(1, 11)):
        url = (
            "https://cosmeet.cosme.net/product/search/srt/4/fw/laneige/itm/800"
            if page == 1
            else base_url.format(page-1)
        )

        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        page_results = extract_cosme_laneige_product_info_from_html(res.text)
        all_results.extend(page_results)

        time.sleep(1) # 서버 부하 방지

    return all_results

laneige_data = product_info()
df_800 = pd.DataFrame(laneige_data)
display(df_800)

,brand,product_name,release_date
0,LANEIGE(ラネージュ),リップスリーピングマスク,2022/9/14
1,LANEIGE(ラネージュ),バウンシースリーピングマスク,2024/5/1
2,LANEIGE(ラネージュ),グレイズ ティントリップセラム,2025/3/1
3,LANEIGE(ラネージュ),リップグロウィバーム,2024/8/30
4,LANEIGE(ラネージュ),ウォータースリーピングマスク N,2025/8/30
5,LANEIGE(ラネージュ),クリームスキン ローション,2023/11/10
6,LANEIGE(ラネージュ),シカスリーピングマスク N,2025/8/30
7,LANEIGE(ラネージュ),バウンシーセラム,2025/4/19
8,LANEIGE(ラネージュ),ワンダーリップデュオキット,2024/11/4
9,LANEIGE(ラネージュ),バウンシーアイスリーピングマスク,2025/3/1


In [19]:
def product_info():
    base_url = "https://cosmeet.cosme.net/product/search/page/{}/srt/4/fw/laneige/itm/803"
    all_results = []
    for page in tqdm(range(1, 11)):
        url = (
            "https://cosmeet.cosme.net/product/search/srt/4/fw/laneige/itm/803"
            if page == 1
            else base_url.format(page-1)
        )

        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if res.status_code != 200:
            break
        page_results = extract_cosme_laneige_product_info_from_html(res.text)
        all_results.extend(page_results)

        time.sleep(1) # 서버 부하 방지

    return all_results


laneige_data = product_info()
df_803 = pd.DataFrame(laneige_data)
display(df_803)

100%|██████████| 10/10 [00:23<00:00,  2.39s/it]


,brand,product_name,release_date
0,LANEIGE(ラネージュ),ラネージュ ネオクッション ミュイ,2025/6/20
1,LANEIGE(ラネージュ),ネオクッション マット N,2023/11/10
2,LANEIGE(ラネージュ),ネオ エッセンシャル フィニッシュパウダー,2024/6/1
3,LANEIGE(ラネージュ),ネオクッション グロウ N,2023/11/10
4,LANEIGE(ラネージュ),スムージーメイクアップセラム,2025/6/20
5,LANEIGE(ラネージュ),ネオ トーンアップ フィニッシュパウダー,2025/6/20
6,LANEIGE(ラネージュ),ラネージュ グロウィ メイクアップセラム,2023/1/11
7,LANEIGE(ラネージュ),ネオファンデーション マット,2021/7/1
8,LANEIGE(ラネージュ),ネオクッション マット メゾンキツネ2022 エコバッグセット,2022/11/2
9,LANEIGE(ラネージュ),ネオクッション マット メゾンキツネ2022 キーリングセット,2022/11/2


In [31]:
def product_info():
    base_url = "https://cosmeet.cosme.net/product/search/page/{}/srt/4/fw/laneige/itm/1005"
    all_results = []
    for page in tqdm(range(1, 4)):
        url = (
            "https://cosmeet.cosme.net/product/search/srt/4/fw/laneige/itm/1005"
            if page == 1
            else base_url.format(page-1)
        )

        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if res.status_code != 200:
            break
        page_results = extract_cosme_laneige_product_info_from_html(res.text)
        all_results.extend(page_results)

        time.sleep(1) # 서버 부하 방지

    return all_results


laneige_data = product_info()
df_1005 = pd.DataFrame(laneige_data)
display(df_1005)

100%|██████████| 3/3 [00:07<00:00,  2.59s/it]


,brand,product_name,release_date
0,LANEIGE(ラネージュ),バウンシースリーピングマスク,2024/5/1
1,LANEIGE(ラネージュ),ウォータースリーピングマスク N,2025/8/30
2,LANEIGE(ラネージュ),シカスリーピングマスク N,2025/8/30
3,LANEIGE(ラネージュ),ウォーターバンク ジェルクリーム,2023/3/4
4,LANEIGE(ラネージュ),ウォーターバンク モイスチャークリーム,2024/11/4
5,LANEIGE(ラネージュ),シカスリーピングマスク,2022/9/14
6,LANEIGE(ラネージュ),ウォーター スリーピング マスク,2023/9/1
7,LANEIGE(ラネージュ),ラネージュ ウォーターバンク クリーム(乾燥肌用),2023/3/4


In [ ]:
def product_info():
    base_url = "https://cosmeet.cosme.net/product/search/page/{}/srt/4/fw/laneige/itm/904"
    all_results = []
    for page in tqdm(range(1, 3)):
        url = (
            "https://cosmeet.cosme.net/product/search/srt/4/fw/laneige/itm/904"
            if page == 1
            else base_url.format(page-1)
        )

        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        page_results = extract_cosme_laneige_product_info_from_html(res.text)
        all_results.extend(page_results)

        time.sleep(1) # 서버 부하 방지

    return all_results


laneige_data = product_info()
df_904 = pd.DataFrame(laneige_data)
display(df_904)

100%|██████████| 2/2 [00:05<00:00,  2.81s/it]


,brand,product_name,release_date
0,LANEIGE(ラネージュ),バウンシースリーピングマスク,2024/5/1
1,LANEIGE(ラネージュ),ウォータースリーピングマスク N,2025/8/30
2,LANEIGE(ラネージュ),シカスリーピングマスク N,2025/8/30
3,LANEIGE(ラネージュ),シカスリーピングマスク,2022/9/14
4,LANEIGE(ラネージュ),ウォーター スリーピング マスク,2023/9/1
